In [3]:
import re
import json
import nltk
import string
import time
import random
from nltk.stem import WordNetLemmatizer, PorterStemmer
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager

# Ensure necessary NLTK resources are downloaded
nltk.download('punkt')
nltk.download('wordnet')

# Initialize lemmatizer and stemmer
lemmer = WordNetLemmatizer()
stemmer = PorterStemmer()

# Tokenization, lemmatization, and stemming functions
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

def StemTokens(tokens):
    return [stemmer.stem(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

# Load responses from JSON
def load_responses(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except Exception as e:
        print(f"Error loading responses: {e}")
        return {}

# Load error responses from a text file
def load_error_responses(file_path):
    try:
        error_responses = {}
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                if line.strip():
                    key, url = line.split(":", 1)
                    error_responses[key.strip()] = url.strip()
        return error_responses
    except Exception as e:
        print(f"Error loading error responses: {e}")
        return {}

# Detect Python or Java errors
def detect_error(user_input):
    tokens = LemNormalize(user_input)
    processed_input = " ".join(tokens)
    
    python_pattern = (
        r"traceback \(most recent call last\):|nameerror|typeerror|valueerror|"
        r"importerror|indentationerror|keyerror|indexerror|attributeerror|"
        r"modulenotfounderror|syntaxerror"
    )
    java_pattern = (
        r"exception in thread.*|nullpointerexception|arrayindexoutofboundsexception|"
        r"classnotfoundexception|illegalargumentexception|numberformatexception|"
        r"filenotfoundexception|noclassdeffounderror|arithmeticexception|"
        r"unsupportedoperationexception|java\.lang\.[a-z]+exception"
    )
    
    if re.search(python_pattern, processed_input, re.IGNORECASE):
        return "python"
    elif re.search(java_pattern, processed_input, re.IGNORECASE):
        return "java"
    return None

# Extract error message
def extract_error_message(error_input):
    match = re.search(r"(nullpointerexception|arrayindexoutofboundsexception|"
                      r"classnotfoundexception|illegalargumentexception|"
                      r"numberformatexception|filenotfoundexception|"
                      r"noclassdeffounderror|arithmeticexception|"
                      r"unsupportedoperationexception|java\.lang\.[a-z]+exception|"
                      r"traceback|nameerror|typeerror|valueerror|"
                      r"importerror|indentationerror|keyerror|indexerror|"
                      r"attributeerror|modulenotfounderror|syntaxerror)", 
                      error_input, re.IGNORECASE)
    if match:
        return match.group(0).lower().replace('java.lang.', '')
    return None

# Search Stack Overflow using Selenium
def search_stack_overflow(error_type, user_input, error_responses):
    error_message = extract_error_message(user_input)
    if not error_message:
        return "Could not identify the error message from the input."

    base_url = error_responses.get(error_type)
    if not base_url:
        return "I couldn't find a suitable Stack Overflow URL for this error type."
    
    search_url = f"{base_url}+{error_message.replace(' ', '+')}"

    try:
        service = Service(GeckoDriverManager().install())
        driver = webdriver.Firefox(service=service)
        driver.get(search_url)

        wait = WebDriverWait(driver, 10)
        results = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.result-link > a')))
        
        if results:
            first_result_url = results[0].get_attribute('href')
            driver.quit()
            return f"Here is a suggested solution: {first_result_url}"
        else:
            driver.quit()
            return "No solutions were found on Stack Overflow for your error."
    except Exception as e:
        return f"An error occurred while accessing Stack Overflow: {str(e)}"

# Get error solution
def get_error_solution(error_type, user_input, error_responses):
    return search_stack_overflow(error_type, user_input, error_responses)

# Chatbot response logic
def response(user_response, responses_data, error_responses):
    if user_response == "patterns":
        patterns = [pattern for intent in responses_data.get("intents", []) for pattern in intent["patterns"]]
        return f"Here are the patterns I recognize: {', '.join(patterns)}"
    
    for intent in responses_data.get("intents", []):
        for pattern in intent["patterns"]:
            if re.search(pattern, user_response, re.IGNORECASE):
                return random.choice(intent["responses"])
    
    error_type = detect_error(user_response)
    if error_type:
        return get_error_solution(error_type, user_response, error_responses)
    
    return "Sorry, I didn't understand that. Can you provide more details?"

# Main chatbot loop
if __name__ == "__main__":
    responses_data = load_responses('responses.json')
    if not responses_data:
        print("Error: Responses data not loaded correctly.")
        exit()

    error_responses = load_error_responses('error_responses.txt')
    if not error_responses:
        print("Error: No error responses found in the text file.")
        exit()

    print("CoxBot: Hello! I can help you with Python or Java errors. Type 'bye' to exit.")

    while True:
        user_response = input("You: ").lower()
        if user_response == 'bye':
            print("CoxBot: Goodbye! Have a nice day!")
            break
        elif user_response in ['thanks', 'thank you']:
            print("CoxBot: You are welcome!")
        else:
            print("CoxBot: ", end="")
            print(response(user_response, responses_data, error_responses))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


CoxBot: Hello! I can help you with Python or Java errors. Type 'bye' to exit.


You:  hi


CoxBot: Hi there, how can I help?


You:  how are you?


...

CoxBot: Sorry, I didn't understand that. Can you provide more details?


You:  who are you?


CoxBot: I am COX


You:  bye


CoxBot: Goodbye! Have a nice day!
